## Databases for each n-gram combination

There are 26 different n-gram combinations with various stride lengths. This notebook constructs a database for each type of n-gram and also a database storing the counts for each n-gram type.

In [1]:
%matplotlib inline

In [3]:
import itertools
import pickle
from collections import defaultdict
import numpy as np
import pickle
import pandas as pd
import os
import time
import dill
from glob import iglob

In [3]:
combinations = []
for n_gram in range(2, 4):
    combinations += [[0] + list(tup) for tup in itertools.combinations(range(1, 7), n_gram-1)]
combinations

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 1, 2],
 [0, 1, 3],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 6],
 [0, 3, 4],
 [0, 3, 5],
 [0, 3, 6],
 [0, 4, 5],
 [0, 4, 6],
 [0, 5, 6]]

In [4]:
combinations = ["".join(str(num) for num in combination) for combination in combinations if combination[-1] == 6]

In [8]:
def getTotalBscore(bscore_file):
    bscore_array = []
    with open(bscore_file,'rb') as f:
        bscore_array = pickle.load(f)
    total_bscore = np.array([]).reshape(62,0)
    for page in bscore_array:
        try:
            total_page = unpackbits(np.array(page), 62)
        except TypeError:
            total_page = np.array([]).reshape(62,0)
            for num in page:
                col = np.array(decodeColumn(num)).reshape(62,-1)
                total_page = np.concatenate((total_page,col),axis=1)
        total_bscore = np.concatenate((total_bscore,total_page),axis=1)
    return total_bscore

In [9]:
def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

In [10]:
def unpackbits(x, num_bits):
    xshape = list(x.shape)
    x = x.reshape([-1, 1])
    mask = 2**np.arange(num_bits, dtype=x.dtype).reshape([1, num_bits])
    return np.flip((x & mask).astype(bool).astype(float), 1).T

We precompute powers of 2 from $2^0$ to $2^{61}$ to speed up calculating hash values.

In [11]:
powers = 1 << np.arange(62)[::-1]

In [12]:
def make_fps(data, combinations, dbs, piece):
    for colindex in range(len(data)):
        for combination in combinations:
            cols = []
            # we need at least enough fingerprints for all the indices in our combination
            try:
                for i in combination:
                    cols.append(data[colindex+int(i)])
            except IndexError:
                continue
            fp = []
            equals_Zero = True
            for column in cols:
                hashint = int(column.dot(powers))
                fp.append(hashint)
                if hashint != 0:
                    equals_Zero = False
            if equals_Zero == True:
                continue
            dbs[combination][tuple(fp)][piece].append(colindex)

In [13]:
def make_DB(filelist, combinations, dbs = None):
    if not dbs:
        dbs = {combination: defaultdict(lambda : defaultdict(list)) for combination in combinations}
    with open(filelist, 'r') as f:
        failed = []
        for i, curfile in enumerate(f):
            curfile = curfile.strip().strip('\n')
            try:
                num = curfile.split('/')[-1][0]
                if num == 'd':
                    data = getTotalBscore(curfile)
                else:
                    with open(curfile, 'rb') as pickle_file:
                        data = pickle.load(pickle_file)
                make_fps(data.T, combinations, dbs, i)
            except:
                failed.append(curfile)
    return dbs

In [14]:
def store_DB(db, combination, outdir):
    with open(f"{outdir}/{combination}.pkl", "wb") as f:
        dill.dump(db, f)

In [15]:
filelist = 'cfg_files/db.list'

In [16]:
db_dir = '/data1/kji/databases_random'

In [14]:
os.makedirs(db_dir, exist_ok=True)

We make a mapping from a number to each piece to save memory.

In [21]:
num_to_piece = {}

In [22]:
with open(filelist, 'r') as f:
    failed = []
    for i, curfile in enumerate(f):
        curfile = curfile.strip().strip('\n')
        piece = curfile.split('/')[-1][:-4]
        num_to_piece[i] = piece

In [25]:
with open("num_to_piece_random.pkl", "wb") as f:
    pickle.dump(num_to_piece, f)

In [17]:
with open("num_to_piece_random.pkl", 'rb') as f:
    num_to_piece = pickle.load(f)

First loop over all combinations (total 26).
Then loop over every single file in IMSLP.
Then within each file, loop over every single offset and compute the n-gram given the combination.
Increment the count for that n-gram combination in the dictionary.
When done, pickle the dictionary and free the memory.

In [20]:
for i in range(0, len(combinations), 3):
    dbs = make_DB(filelist, combinations[i: i+3])
    for combination in dbs:
        store_DB(dbs[combination], combination, db_dir)
    dbs.clear()

In [40]:
dbs = make_DB(filelist, ['06'])
for combination in dbs:
    store_DB(dbs[combination], combination, db_dir)
dbs.clear()

## Construct database of counts

This database is a mapping from each unique n-gram that is one of our 26 types to the number of matches in all of IMSLP.

In [12]:
files = []
for filename in iglob(f"{db_dir}/*.pkl", recursive=True):
    files.append(filename)

In [13]:
files.sort(key = len)

In [ ]:
def store_DB(db, combination, outdir):
    with open(f"{outdir}/{combination}.pkl", "wb") as f:
        pickle.dump(db, f)

In [ ]:
filename = f"/data1/kji/databases/split21_Bt_200mill.pkl"
with open(filename, "rb") as f:
    d = pickle.load(f)
counts = {}
for fp in d:
    counts[fp] = sum([len(d[fp][piece]) for piece in d[fp]])
d.clear()
print(f"finished processing {filename}")
store_DB(counts, f"split21_Bt_200mill_counts", db_dir)
print(f"stored database for {filename}")

In [9]:
store_DB(counts, f"split21_Bt_200mill_counts", db_dir)

In [ ]:
for combination in combinations:
    filename = f"/data1/kji/databases2/{combination}.pkl"
    with open(filename, "rb") as f:
        d = dill.load(f)
        counts = {}
        for fp in d:
            counts[fp] = sum([len(d[fp][piece]) for piece in d[fp]])
        d.clear()
        print(f"finished processing {filename}")
        f.flush()
        store_DB(counts, f"{combination}_counts", db_dir)
        print(f"stored database for {filename}")

In [ ]:
store_DB(matches, "matches_array", db_dir)

In [ ]:
for combination in combinations:
    filename = f"/data1/kji/databases/{combination}_counts.pkl"
    with open(filename, "rb") as f:
        d = pickle.load(f)
        length = len(d)
        fps = np.empty(length, dtype = object)
        values = np.empty(length, dtype = int)
        for i, (n_gram, count) in enumerate(d.items()):
            fps[i] = n_gram
            values[i] = count
        d.clear()
        n_grams = np.concatenate([n_grams, fps])
        matches = np.concatenate([matches, values])
        print(f"finished processing {filename}")
        f.flush()

finished processing /data1/kji/databases/0125_counts.pkl
finished processing /data1/kji/databases/0134_counts.pkl
finished processing /data1/kji/databases/0135_counts.pkl
finished processing /data1/kji/databases/0145_counts.pkl
finished processing /data1/kji/databases/0234_counts.pkl


## Make fp matches table

Get the number of PDFs that contain each fingerprint.

In [5]:
db_dir = "/data1/kji/databases_random"

In [31]:
with open("/data1/kji/databases_v4/fp_matches.pkl", "rb") as f:
    fp_matches = pickle.load(f)

In [6]:
fp_matches = {combination: {} for combination in combinations}

In [7]:
for combination in combinations:
    with open(f"{db_dir}/{combination}.pkl", "rb") as f:
        d = dill.load(f)
    for fp in d:
        # first element is number of times the fingerprint occurs, second element is number of PDFs containing the fingerprint
        fp_matches[combination][fp] = (sum(len(d[fp][piece]) for piece in d[fp]), len(d[fp]))
    d.clear()

In [8]:
with open(f"/data1/kji/databases_v5/fp_matches.pkl", "wb") as f:
    pickle.dump(fp_matches, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open(f"/data1/kji/databases_v4/fp_matches.pkl", "wb") as f:
    pickle.dump(fp_matches, f, protocol=pickle.HIGHEST_PROTOCOL)